In [6]:
import pandas as pd
file_path = 'recipes_ingredients_grouped_full.xlsx'
df1 = pd.read_excel(file_path)

In [7]:
import requests
import time  # Thêm thư viện time để thực hiện delay

# Thông tin API của bạn từ Edamam
APP_ID = '2c38c482'
APP_KEY = '5267666a25015ecc1d55951cf04e0ed2'

# Hàm để lấy thông tin dinh dưỡng từ Edamam API với ingredient, amount, và unit (nếu có)
def get_nutrition_data_from_edamam(ingredient, amount, unit):
    # Xử lý trường hợp thiếu `amount` hoặc `unit`
    if pd.isna(amount) and pd.isna(unit):
        query = f"{ingredient}"  # Chỉ sử dụng ingredient
    elif pd.isna(amount):  # Nếu thiếu `amount`, chỉ dùng `unit` và `ingredient`
        query = f"{unit} {ingredient}"
    elif pd.isna(unit):  # Nếu thiếu `unit`, chỉ dùng `amount` và `ingredient`
        query = f"{amount} {ingredient}"
    else:
        query = f"{amount} {unit} {ingredient}"  # Đủ cả `amount`, `unit` và `ingredient`
    
    # URL cho API của Edamam
    url = f"https://api.edamam.com/api/nutrition-data?app_id={APP_ID}&app_key={APP_KEY}&ingr={query}"
    
    # In ra thông báo bắt đầu quá trình lấy dữ liệu cho nguyên liệu này
    print(f"Đang lấy thông tin dinh dưỡng cho: {query}")
    
    # Gửi yêu cầu GET tới API của Edamam
    response = requests.get(url)
    
    # Kiểm tra xem yêu cầu có thành công không
    if response.status_code == 200:
        data = response.json()
        
        # Kiểm tra xem có kết quả nào không
        if 'calories' in data:
            calories = data.get('calories', 0)  # Calories
            totalNutrients = data.get('totalNutrients', {})
            protein = totalNutrients.get('PROCNT', {}).get('quantity', 0)  # Protein
            carbohydrate = totalNutrients.get('CHOCDF', {}).get('quantity', 0)  # Carbohydrate
            fat = totalNutrients.get('FAT', {}).get('quantity', 0)  # Fat
            
            # Làm tròn các giá trị dinh dưỡng đến 5 chữ số thập phân
            calories = round(calories, 5)
            protein = round(protein, 5)
            carbohydrate = round(carbohydrate, 5)
            fat = round(fat, 5)
            
            # In ra thông tin dinh dưỡng đã nhận được
            print(f"Nhận được: Calories = {calories}, Protein = {protein}, Carbohydrate = {carbohydrate}, Fat = {fat}\n")
            
            return calories, protein, carbohydrate, fat
        else:
            # In ra nếu không tìm thấy kết quả
            print(f"Không tìm thấy thông tin dinh dưỡng cho: {query}\n")
    elif response.status_code == 429:
        # Nếu gặp lỗi 429 (Too Many Requests), đợi vài giây trước khi thử lại
        print(f"Lỗi 429: Quá nhiều yêu cầu. Đang chờ 10 giây trước khi thử lại...")
        time.sleep(20)  # Chờ 20 giây trước khi thử lại
        return get_nutrition_data_from_edamam(ingredient, amount, unit)  # Thử lại yêu cầu
    else:
        # In ra nếu có lỗi khác trong quá trình gửi yêu cầu
        print(f"Lỗi {response.status_code} khi lấy thông tin cho: {query}\n")
    
    # Trả về None nếu không tìm thấy
    return None, None, None, None

# Thêm các cột dinh dưỡng mới vào dataset
df1['calories'] = None
df1['protein'] = None
df1['carbohydrate'] = None
df1['fat'] = None

# Hàm để thêm thông tin dinh dưỡng vào dataset
def add_nutrition_data(row):
    # Lấy thông tin từ các cột `ingredient`, `amount`, và `unit`
    ingredient = row['Ingredient Name']
    amount = row['Amount']
    unit = row['Unit']
    
    # Gọi API với ingredient, amount, và unit (nếu có)
    calories, protein, carbs, fat = get_nutrition_data_from_edamam(ingredient, amount, unit)
    
    # Thêm các giá trị dinh dưỡng vào từng cột
    row['calories'] = calories
    row['protein'] = protein
    row['carbohydrate'] = carbs
    row['fat'] = fat
    return row

# Áp dụng hàm để lấy và thêm dữ liệu dinh dưỡng vào từng dòng
df1 = df1.apply(add_nutrition_data, axis=1)

Đang lấy thông tin dinh dưỡng cho: 4 cups (100g) chopped kale, stems removed
Nhận được: Calories = 35, Protein = 2.92, Carbohydrate = 4.42, Fat = 1.49

Đang lấy thông tin dinh dưỡng cho: 2 cups (50g) fresh basil leaves
Nhận được: Calories = 11, Protein = 1.575, Carbohydrate = 1.325, Fat = 0.32

Đang lấy thông tin dinh dưỡng cho: 2 jalapeños (about 28g), seeds removed
Nhận được: Calories = 7, Protein = 0.23442, Carbohydrate = 1.6744, Fat = 0.09531

Đang lấy thông tin dinh dưỡng cho: 1 cup (170g) chopped zucchini
Nhận được: Calories = 28, Protein = 2.057, Carbohydrate = 5.287, Fat = 0.544

Đang lấy thông tin dinh dưỡng cho: 2 celery ribs (about 80g)
Nhận được: Calories = 9, Protein = 0.49128, Carbohydrate = 2.11464, Fat = 0.12104

Đang lấy thông tin dinh dưỡng cho: 3 pitted medjool dates (about 75g)
Nhận được: Calories = 191, Protein = 1.2489, Carbohydrate = 51.75, Fat = 0.1035

Đang lấy thông tin dinh dưỡng cho: 2 tablespoons hemp seeds
Nhận được: Calories = 170, Protein = 7.44, Carbohy

In [9]:
df1.to_excel('nutrition_facts.xlsx')

In [10]:
# Từ điển chuẩn hóa các đơn vị đo lường
unit_synonyms = {
    "tablespoons": "tbsp",
    "tablespoon": "tbsp",
    "teaspoons": "tsp",
    "teaspoon": "tsp",
    "pounds": "lb",
    "pound": "lb", 
    "ounces": "oz",
    "ounce": "oz"
}

# Hàm để chuẩn hóa đơn vị đo lường
def standardize_unit(unit):
    if unit in unit_synonyms:
        return unit_synonyms[unit]
    return unit  # Giữ nguyên nếu không có trong từ điển

# Áp dụng hàm chuẩn hóa đơn vị cho cột 'unit'
df1['Unit'] = df1['Unit'].apply(standardize_unit)

In [11]:
df1

Unnamed: 0                Recipe Name Ingredient Group  \
0              0     Kale Pesto Basil Wraps       Green Wrap   
1              1     Kale Pesto Basil Wraps       Green Wrap   
2              2     Kale Pesto Basil Wraps       Green Wrap   
3              3     Kale Pesto Basil Wraps       Green Wrap   
4              4     Kale Pesto Basil Wraps       Green Wrap   
...          ...                        ...              ...   
1231        1231  Roasted Vegetable Lasagna              NaN   
1232        1232  Roasted Vegetable Lasagna              NaN   
1233        1233  Roasted Vegetable Lasagna              NaN   
1234        1234  Roasted Vegetable Lasagna              NaN   
1235        1235  Roasted Vegetable Lasagna              NaN   

                           Ingredient Name Amount  Unit  \
0       (100g) chopped kale, stems removed      4  cups   
1                 (50g) fresh basil leaves      2  cups   
2     jalapeños (about 28g), seeds removed      2   NaN   
3                  (170g) chopped zucchini      1   cup   
4                  celery ribs (about 80g)      2   NaN   
...                                    ...    ...   ...   
1231                          onion powder      1   tsp   
1232                     red pepper flakes      ¼   tsp   
1233                   ground black pepper      ¼   tsp   
1234     jarred or homemade marinara sauce      3  cups   
1235                            Nutty Parm      ¼   cup   

                             ingredient  processed  preliminary_processed  \
0                          chopped kale      False                   True   
1                    fresh basil leaves      False                  False   
2                             jalapeños      False                  False   
3                      chopped zucchini      False                   True   
4                           celery ribs      False                  False   
...                                 ...        ...                    ...   
1231                       onion powder       True                  False   
1232                  red pepper flakes       True                  False   
1233                ground black pepper       True                  False   
1234  jarred or homemade marinara sauce       True                  False   
1235                         Nutty Parm      False                  False   

      fresh  ...  dried  optional  garnish  for_dipping  to_taste  \
0     False  ...  False     False    False        False     False   
1      True  ...  False     False    False        False     False   
2     False  ...  False     False    False        False     False   
3     False  ...  False     False    False        False     False   
4     False  ...  False     False    False        False     False   
...     ...  ...    ...       ...      ...          ...       ...   
1231  False  ...  False     False    False        False     False   
1232  False  ...  False     False    False        False     False   
1233  False  ...  False     False    False        False     False   
1234  False  ...  False     False    False        False     False   
1235  False  ...  False     False    False        False     False   

      different_choices  calories   protein  carbohydrate       fat  
0                 False        35   2.92000       4.42000   1.49000  
1                 False        11   1.57500       1.32500   0.32000  
2                 False         7   0.23442       1.67440   0.09531  
3                 False        28   2.05700       5.28700   0.54400  
4                 False         9   0.49128       2.11464   0.12104  
...                 ...       ...       ...           ...       ...  
1231              False         8   0.24960       1.89840   0.02496  
1232              False         0   0.02043       0.13470   0.01120  
1233              False         1   0.05980       0.36800   0.01874  
1234              False       396  11.00880      58.84560  12.75120  
1235

In [13]:
df1.to_excel('nutrition_facts_full.xlsx') 